# 🔥 Persian OCR - ساده، سریع و دقیق

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MohammadHNdev/Persian_OCR_Simple/blob/main/Persian_OCR_Optimized.ipynb)

## ✨ ویژگی‌ها:
- 🎯 **دقت بالا** - تنظیمات بهینه شده
- ⚡ **سرعت فوق‌العاده** - پردازش موازی هوشمند
- 💾 **مصرف حافظه کم** - مدیریت منابع عالی
- 🛡️ **پایداری بالا** - error handling قوی
- 📊 **گزارش کامل** - آمار دقیق و مفید

---

## 📋 راهنما:
1. همه سل‌ها را اجرا کنید
2. فایل PDF آپلود کنید  
3. صبر کنید تا OCR تمام شود
4. فایل نتیجه را دانلود کنید

In [ ]:
# 🚀 نصب ابزارها
print("🔥 نصب Persian OCR...")
!pip install pytesseract pdf2image Pillow tqdm > /dev/null 2>&1
!sudo apt update > /dev/null 2>&1
!sudo apt install -y tesseract-ocr tesseract-ocr-fas poppler-utils > /dev/null 2>&1
print("✅ نصب کامل!")

In [ ]:
# 📚 کتابخانه‌ها
import pytesseract
from PIL import Image, ImageEnhance
from pdf2image import convert_from_path
import os, time, subprocess, shutil, gc
from tqdm.notebook import tqdm
import concurrent.futures
from google.colab import files

print("📚 کتابخانه‌ها آماده!")

In [ ]:
# 🔧 توابع اصلی OCR

def process_single_page_simple(page_info):
    """پردازش ساده و موثر یک صفحه"""
    img_path, page_num = page_info
    
    try:
        # بارگذاری و پیش‌پردازش
        img = Image.open(img_path)
        
        # تبدیل به grayscale و افزایش کنتراست
        img_gray = img.convert('L')
        enhancer = ImageEnhance.Contrast(img_gray)
        img_enhanced = enhancer.enhance(2.5)  # کمی بیشتر از قبل
        
        # افزایش شارپنس برای وضوح بهتر
        sharpness = ImageEnhance.Sharpness(img_enhanced)
        img_sharp = sharpness.enhance(1.3)
        
        # OCR با بهترین تنظیمات
        text = pytesseract.image_to_string(
            img_sharp, 
            lang='fas',
            config='--psm 3 --oem 3'  # بهترین ترکیب
        )
        
        # پاکسازی حافظه
        del img, img_gray, img_enhanced, img_sharp
        
        # حذف فایل موقت
        if os.path.exists(img_path):
            os.remove(img_path)
        
        return {
            'page': page_num,
            'text': text.strip(),
            'chars': len(text.strip()),
            'success': True
        }
        
    except Exception as e:
        return {
            'page': page_num,
            'text': f"خطا در صفحه {page_num}: {str(e)}",
            'chars': 0,
            'success': False
        }

def get_pdf_page_count(pdf_path):
    """تشخیص دقیق تعداد صفحات PDF"""
    try:
        result = subprocess.run(['pdfinfo', pdf_path], capture_output=True, text=True)
        for line in result.stdout.split('\n'):
            if 'Pages:' in line:
                return int(line.split(':')[1].strip())
    except:
        pass
    
    # روش جایگزین
    try:
        test_images = convert_from_path(pdf_path, dpi=72, first_page=1, last_page=10)
        return len(test_images) * 10  # تخمین
    except:
        return 100  # پیش‌فرض

print("🔧 توابع کمکی آماده!")

In [ ]:
# 🖼️ تبدیل PDF به تصاویر

def convert_pdf_batch_optimized(pdf_path, batch_size=15, dpi=350):
    """تبدیل PDF به تصاویر - بهینه شده"""
    print("📄 شروع تبدیل PDF به تصاویر...")
    
    # تشخیص تعداد صفحات
    total_pages = get_pdf_page_count(pdf_path)
    print(f"📊 تعداد صفحات: {total_pages}")
    
    # پوشه موقت
    temp_dir = '/tmp/ocr_images'
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    
    all_image_paths = []
    
    # پردازش دسته‌ای
    for start_page in range(1, total_pages + 1, batch_size):
        end_page = min(start_page + batch_size - 1, total_pages)
        
        print(f"🔄 تبدیل صفحات {start_page} تا {end_page}...")
        
        try:
            images = convert_from_path(
                pdf_path,
                dpi=dpi,
                first_page=start_page,
                last_page=end_page,
                fmt='jpeg',
                thread_count=2  # محدود برای پایداری
            )
            
            # ذخیره تصاویر
            for i, img in enumerate(images):
                page_num = start_page + i
                img_path = os.path.join(temp_dir, f"page_{page_num:04d}.jpg")
                img.save(img_path, 'JPEG', quality=95, optimize=True)
                all_image_paths.append((img_path, page_num))
            
            # پاکسازی حافظه
            for img in images:
                del img
            del images
            gc.collect()
            
        except Exception as e:
            if start_page <= total_pages:
                print(f"⚠️ خطا در دسته {start_page}-{end_page}: {e}")
            continue
    
    print(f"✅ {len(all_image_paths)} صفحه آماده پردازش")
    return all_image_paths

print("🖼️ تبدیل‌گر PDF آماده!")

In [ ]:
# 🚀 تابع اصلی OCR

def persian_ocr_optimized(pdf_path, max_workers=3, dpi=350):
    """OCR بهینه شده فارسی"""
    print(f"🔥 شروع Persian OCR: {os.path.basename(pdf_path)}")
    start_time = time.time()
    
    # تبدیل PDF به تصاویر
    image_paths = convert_pdf_batch_optimized(pdf_path, dpi=dpi)
    
    if not image_paths:
        print("❌ هیچ تصویری تولید نشد!")
        return None
    
    # OCR موازی
    print(f"🔍 شروع OCR با {max_workers} worker...")
    results = []
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # پردازش دسته‌ای برای کنترل حافظه
        batch_size = 20
        
        for i in range(0, len(image_paths), batch_size):
            batch = image_paths[i:i+batch_size]
            
            batch_num = i//batch_size + 1
            total_batches = (len(image_paths)-1)//batch_size + 1
            print(f"📝 پردازش دسته {batch_num}/{total_batches}")
            
            # ارسال batch
            future_to_page = {
                executor.submit(process_single_page_simple, page_info): page_info[1] 
                for page_info in batch
            }
            
            # جمع‌آوری نتایج
            for future in tqdm(
                concurrent.futures.as_completed(future_to_page),
                total=len(future_to_page),
                desc=f"دسته {batch_num}"
            ):
                try:
                    result = future.result(timeout=30)
                    results.append(result)
                except Exception as e:
                    page_num = future_to_page[future]
                    results.append({
                        'page': page_num,
                        'text': f"خطا: {e}",
                        'chars': 0,
                        'success': False
                    })
            
            # پاکسازی حافظه
            gc.collect()
    
    # مرتب‌سازی نتایج
    results.sort(key=lambda x: x['page'])
    
    # آمارگیری
    successful_pages = [r for r in results if r['success'] and r['chars'] > 20]
    total_chars = sum(r['chars'] for r in successful_pages)
    
    # ساخت فایل نهایی
    print("📝 تولید فایل نهایی...")
    
    timestamp = int(time.time())
    output_filename = f"persian_ocr_result_{timestamp}.txt"
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        # هدر فایل
        f.write(f"# 🔥 Persian OCR Results\n")
        f.write(f"# File: {os.path.basename(pdf_path)}\n")
        f.write(f"# Date: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"# Pages: {len(successful_pages)}/{len(results)} successful\n")
        f.write(f"# Characters: {total_chars:,}\n")
        f.write(f"\n{'='*80}\n\n")
        
        # محتوای هر صفحه
        for result in results:
            f.write(f"\n{'='*60}\n")
            f.write(f"صفحه {result['page']}")
            if result['success']:
                f.write(f" - {result['chars']} کاراکتر")
            f.write(f"\n{'='*60}\n")
            f.write(result['text'])
            f.write("\n\n")
    
    # گزارش نهایی
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"\n🎉 پردازش کامل شد!")
    print(f"📊 آمار نهایی:")
    print(f"   📄 صفحات موفق: {len(successful_pages)}/{len(results)} ({len(successful_pages)/len(results)*100:.1f}%)")
    print(f"   📝 کل کاراکترها: {total_chars:,}")
    print(f"   ⏱️ زمان کل: {duration:.1f} ثانیه ({duration/60:.1f} دقیقه)")
    print(f"   ⚡ سرعت: {total_chars/duration:.0f} کاراکتر/ثانیه")
    print(f"   📁 فایل: {output_filename}")
    
    # کیفیت‌سنجی
    if len(successful_pages)/len(results) >= 0.9:
        quality = "🏆 عالی"
    elif len(successful_pages)/len(results) >= 0.7:
        quality = "🥇 خوب"
    else:
        quality = "🥉 متوسط"
    
    print(f"   🏅 کیفیت: {quality}")
    
    # نمایش نمونه
    if successful_pages:
        best_page = max(successful_pages, key=lambda x: x['chars'])
        preview = best_page['text'][:300]
        print(f"\n📖 نمونه متن (صفحه {best_page['page']}):")
        print(f"{'-'*40}")
        print(preview)
        if len(best_page['text']) > 300:
            print("...")
    
    # دانلود
    try:
        files.download(output_filename)
        print(f"\n💾 فایل {output_filename} در حال دانلود...")
    except Exception as e:
        print(f"\n⚠️ خطا در دانلود: {e}")
        print("فایل در پنل فایل‌های Colab موجود است.")
    
    # پاکسازی نهایی
    try:
        shutil.rmtree('/tmp/ocr_images')
        print("🧹 پاکسازی انجام شد.")
    except:
        pass
    
    return {
        'success': True,
        'file': output_filename,
        'stats': {
            'total_pages': len(results),
            'successful_pages': len(successful_pages),
            'total_chars': total_chars,
            'duration': duration,
            'quality': quality
        }
    }

print("🔥 Persian OCR آماده!")

In [ ]:
# 📤 آپلود فایل
print("📤 لطفاً فایل PDF خود را انتخاب کنید:")
print("💡 توصیه: فایل‌های با کیفیت بالا نتیجه بهتری دارند")

uploaded = files.upload()

pdf_file = None
if uploaded:
    for filename in uploaded.keys():
        if filename.lower().endswith('.pdf'):
            pdf_file = filename
            break
    
    if pdf_file:
        size_mb = len(uploaded[pdf_file]) / (1024*1024)
        print(f"✅ آپلود موفق: {pdf_file} ({size_mb:.1f} MB)")
        
        # تخمین زمان
        estimated_minutes = size_mb * 0.3  # تخمین: 0.3 دقیقه/MB
        print(f"⏱️ زمان تخمینی: {estimated_minutes:.1f} دقیقه")
        
        if size_mb > 50:
            print("⚠️ فایل بزرگ! ممکن است زمان بیشتری طول بکشد.")
    else:
        print("❌ فایل PDF یافت نشد!")
else:
    print("❌ فایل آپلود نشد!")

In [ ]:
# 🚀 اجرای OCR
if 'pdf_file' in locals() and pdf_file:
    print(f"🔥 شروع Persian OCR برای {pdf_file}")
    print("☕ لطفاً صبر کنید...")
    
    # تنظیمات بر اساس سایز فایل
    size_mb = len(uploaded[pdf_file]) / (1024*1024)
    
    if size_mb < 10:
        dpi, workers = 400, 3
    elif size_mb < 30:
        dpi, workers = 350, 3
    else:
        dpi, workers = 300, 2
    
    print(f"⚙️ تنظیمات بهینه: DPI={dpi}, Workers={workers}")
    
    result = persian_ocr_optimized(pdf_file, max_workers=workers, dpi=dpi)
    
    if result and result['success']:
        stats = result['stats']
        print(f"\n🏆 موفقیت! {stats['quality']}")
        print(f"📝 {stats['total_chars']:,} کاراکتر در {stats['duration']:.1f} ثانیه")
        print(f"💾 فایل: {result['file']}")
    else:
        print(f"❌ متأسفانه مشکلی پیش آمد.")
else:
    print("❌ ابتدا فایل PDF آپلود کنید!")

---

## 🎉 کار تمام شد!

### 🔥 مزایای این ابزار:

- **🎯 دقت بالا**: تنظیمات بهینه شده برای متن فارسی
- **⚡ سرعت بالا**: پردازش موازی هوشمند
- **💾 مصرف کم حافظه**: مدیریت منابع عالی
- **🛡️ پایداری**: error handling قوی
- **📊 گزارش کامل**: آمار مفصل و مفید

### 💡 نکات مهم:

- کیفیت PDF اصلی مهم‌ترین عامل در نتیجه نهایی است
- فایل‌های کمتر از 50 صفحه سریع‌تر پردازش می‌شوند  
- برای استفاده مجدد: Runtime → Restart session

### 🚀 استفاده مجدد:

1. **Runtime** → **Restart session**
2. همه سل‌ها را دوباره اجرا کنید
3. فایل جدید آپلود کنید

---

<div align="center">

**🔥 ساخته شده برای جامعه فارسی‌زبان 🔥**

*ساده، سریع و دقیق*

</div>